In [1]:
%load_ext pretty_jupyter

In [2]:
import os,re
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd
from itertools import product
from datetime import datetime

import matplotlib.pyplot as plt
from pretty_jupyter.helpers import matplotlib_fig_to_html, matplotlib_fig_to_markdown
from IPython.display import display, Markdown

import seaborn as sns
from sklearn.linear_model import LinearRegression

import geopandas
import plotly.express as px

import plot_crime as pc
import plot_crime_plotly as pcp

%load_ext autoreload
%autoreload 2
# %matplotlib inline
# %load_ext genai

In [3]:
import plotly.io as pio
# pio.renderers.default = "notebook_connected"
# pio.renderers
# pio.renderers.default = "browser"

In [4]:
include_plotlyjs = 'cdn'
full_html = False
renderer=None#"notebook_connected"

In [5]:
def get_region_df(plot_data,region):
    select_places = community_areas.query('Region == @region')['Name'].values
    return plot_data.query('`Community Area` in @select_places')

In [6]:
base_dir = 'C:\\Users\\BatLab\\Documents\\chicago_crime'
crime_by_year_fname = os.path.join(base_dir,'crime_by_year_dfs.pkl')
crime_by_year_dfs = pd.read_pickle(crime_by_year_fname)

return_raw = False
community_areas, demographics, nibrs_codes, iucr_codes, population, _ = pc.init_crime_data(return_raw)

cca_regions = community_areas.Region.unique()

# Crime rates by year

All of the following plots show crime rates by year within one of the 9 regions or one of the 77 neighborhoods of Chicago.

Crime type is categorized using FBI standardized types.

**Plots are interactive**

- Hover over lines to get more details (*including raw totals*). 
- Single click on a legend entry to remove that entry from plots
- Double click on a legend entry to remove all other entries
- Pan, zoom, and reset axes using button on top right of figure

In [ ]:
crime_type = 'FBI_type'
place_type = 'region'
select_crimes = None
select_places = cca_regions

## Plotting each crime type per region

The following plot show the yearly **per-capita** crime rate within one of 9 regions of Chicago.

In [ ]:
plot_data = pcp.melt_crime_df(
    crime_by_year_dfs,
    crime_type,
    place_type,
    select_crimes=select_crimes,
    population=population,
    select_places=select_places,
    community_areas=community_areas
)

fig = pcp.plot_crime_by_type_and_place(
    plot_data,
    crime_type,
    place_type
)
fig.show(renderer=renderer)

In [ ]:
%%jmd
## Plotting Crimes within Region by Neighborhood
[//]: # (-.- .tabset .tabset-pills)

In this section, use the tabs at the top to select which **region** to display. 

Within those regions, per-capita crime rates are plotted separately by the individual **neighborhoods** making up that region.


In [ ]:
crime_type = 'FBI_type'
place_type = 'Community Area'
select_crimes = None
select_places = community_areas.index
select_regions = [r for r in pcp.ordered_regions if r is not None]

In [ ]:
plot_data = pcp.melt_crime_df(
    crime_by_year_dfs,
    crime_type,
    place_type,
    select_crimes=select_crimes,
    population=population,
    select_places=select_places,
    community_areas=community_areas
)

for region in select_regions:

    display(Markdown(f"### {region.title()}"))
    plot_data_reg = get_region_df(plot_data,region) 
    fig = pcp.plot_crime_by_type_and_place(
            plot_data_reg,
            crime_type,
            place_type
        )
    
    fig.show(renderer=renderer)

In [ ]:
%%jmd
## Plotting Select Crimes by Region
[//]: # (-.- .tabset .tabset-pills)
    
- In this section, use the tabs at the top to select which grouping of crime types to display.

- Groupings are arbitrarily assigned. 
- "Society" is a grab-all grouping.

- Regions are arranged semi-geographically.

Instead of the per-capita rate, these plots show the **fraction of the highest recorded crime count** for each given crime type and region:

- For instance, on the Far North Side, homicides reached their maximum in 2022. So each data point shows how many homicides were commited in the Far North Side that year as a fraction of the number that occurred in 2022. 

- This scaling allows trends to be compared when total counts/rates are dissimilar between crime types.


In [ ]:
crime_type = 'FBI_type'
place_type = 'region'
select_places = None
select_crimes = None

In [ ]:

for select_crime_type in pcp.select_crime_lists:
    display(Markdown(f"### {select_crime_type.replace('_',' ').title()}"))

    plot_data = pcp.melt_crime_df(
        crime_by_year_dfs,
        crime_type,
        place_type,
        select_crimes=select_crime_type
    )
    fig = pcp.plot_regional_crime(
            plot_data,
            crime_type,
            select_crimes=select_crime_type
    )

    fig.show(renderer=renderer)

# Mapping crime rates by year

This section geographically maps per-capita crime rates by year using the 77 neighborhoods of Chicago.

Crime type is categorized using FBI standardized types.

**Plots are interactive**

- Hover over lines to get more details (*including raw totals*). 
- Pan, zoom, and reset axes using button on top right of figure

In [7]:
%%jmd
## Mapping Crimes by Neighborhood
[//]: # (-.- .tabset .tabset-pills)
    
- In this section, use the tabs at the top to select which type of crime to display.


## Mapping Crimes by Neighborhood
<span class='pj-token' style='display: none;'>.tabset|O_O|.tabset-pills</span>
    
- In this section, use the tabs at the top to select which type of crime to display.

In [8]:
crime_type = 'FBI_type'
place_type = 'Community Area'
select_crimes = None
select_places = community_areas.index
select_regions = [r for r in pcp.ordered_regions if r is not None]

In [22]:
plot_data = pcp.melt_crime_df(
    crime_by_year_dfs,
    crime_type,
    place_type,
    select_crimes=select_crimes,
    population=population,
    select_places=select_places,
    community_areas=community_areas
)

In [27]:
map_df = pcp.get_map_df(tol=5e-4)

In [ ]:

for crime_type in pcp.used_fbi_types:
    display(Markdown(f"### {crime_type.replace('_',' ').title()}"))
    
    fig = pcp.map_crime_by_place(
        map_df,
        plot_data,
        place_type,
        crime_type
    )
    
    fig.show(renderer=renderer)